# Problem

In the three-period model of Figure 1.2.2 of Chapter 1, let the interest rate be $r=\frac{1}{4}$ so the risk-neutral probabilities are $\tilde{p}=\tilde{q}=\frac{1}{2}$.  Find the time-zero price and optimal exercise policy (optimal stopping time) for the path-dependent American derivative security whose intrinsic value at each time $n$, $n=0,1,2,3$, is $\left(4-\frac{1}{n+1}\sum_{j=0}^nS_j\right)^{+}$.  This intrinsic value is a put on the average stock price between time zero and time $n$.

# Answer

According to Theorem 4.4.5, the optimal stopping time is given by
\begin{equation}
\tau^{*}(\omega_1\dots\omega_{n}\omega_{n+1}\dots\omega_N)=\min\{n;\;V_n(\omega_1\dots\omega_n)=G_n(\omega_1\dots\omega_n)\}.
\end{equation}

To find these values of $\tau^{*}(\omega_1\omega_2\omega_3)$, we first compute all the values of $G_t(.)$ and $V_t(.)$ at each time step, and for every possible coin toss.  See code below.

In [1]:
# returns list of possible sequence of coin tosses of length n.
def possible_coin_tosses(n=1):
    coin_toss_possibilities = []
    for bin_seq in [(n-len(bin(i)[2:]))*'0'+bin(i)[2:] for i in range(2**n)]:
        coin_toss_seq = ''
        for bit in bin_seq:
            if bit == '0':
                coin_toss_seq = coin_toss_seq + 'H'
            elif bit == '1':
                coin_toss_seq = coin_toss_seq + 'T'
        coin_toss_possibilities.append(coin_toss_seq)
    return coin_toss_possibilities

In [2]:
# based on constant d, r, and u factors, generate binomial model
# list of stock prices, given some initial stock price,
# from time 0 to time t_f, inclusive
def bm_stock_prices(s_0=4, d=0.5, r=0.25, u=2, t_f=3):
    S={0:{'': s_0}}
    for t in range(1,t_f+1):
        S[t]={}
        for coin_toss_seq in possible_coin_tosses(t):
            mult_factor = 1
            for toss in coin_toss_seq:
                if toss == 'H':
                    mult_factor *= u
                elif toss == 'T':
                    mult_factor *= d
            S[t][coin_toss_seq]=s_0*mult_factor
    return S

In [3]:
S = bm_stock_prices(s_0=4, d=0.5, r=0.25, u=2, t_f=3)

In [4]:
# intrinsic value for the american derivative under consideration
def g(s, coin_toss_seq):
    # initialize average
    avg = s[0]['']
    for i in range(1, len(coin_toss_seq)+1):
        partial_coin_toss_seq = coin_toss_seq[:i]
        avg += s[i][partial_coin_toss_seq]
    avg = avg/(len(coin_toss_seq)+1)
    return s[0]['']-avg

In [5]:
# Calculate intrinsic value for all price paths
N = 3
G = {t:{} for t in range(0,N+1)}
for t in S.keys():
    for coin_toss_seq in S[t].keys():
        G[t][coin_toss_seq] = g(S, coin_toss_seq)

In [6]:
# parameters of this binomial model
d = 0.5
r = 0.25
u = 2
p = (1+r-d)/(u-d)
q = (u-1-r)/(u-d)

In [7]:
# Calculate derivative price for all paths using Theorem 4.4.3
# Based on the stock prices in the dictionary S and the intrinsic values in
# the dictionary G
V = {}
for t in list(S.keys())[::-1]:
    V[t] = {}
    for coin_toss_seq, value in G[t].items():
        if t==max(S.keys()):
            V[t][coin_toss_seq] = max([value, 0])
        else:
            rnd_price = (1/(1+r))*(p*V[t+1][coin_toss_seq+'H']+q*V[t+1][coin_toss_seq+'T'])
            V[t][coin_toss_seq] = max([value, rnd_price])     
print(V)

{3: {'HHH': 0, 'HHT': 0, 'HTH': 0, 'HTT': 0, 'THH': 0, 'THT': 1.0, 'TTH': 1.75, 'TTT': 2.125}, 2: {'HH': 0.0, 'HT': 0.0, 'TH': 0.6666666666666665, 'TT': 1.6666666666666665}, 1: {'H': 0.0, 'T': 1.0}, 0: {'': 0.4}}


According to Theorem 4.4.5, the optimal stopping time is given by
\begin{equation}
\tau^{*}(\omega_1\dots\omega_{n}\omega_{n+1}\dots\omega_N)=\min\{n;\;V_n(\omega_1\dots\omega_n)=G_n(\omega_1\dots\omega_n)\}.
\end{equation}

In [8]:
# return n given above, given dictionary V, G, and coin_toss_seq of length N
def min_n(v, g, coin_toss_seq):
    for t in range(len(coin_toss_seq)):
        partial_seq = coin_toss_seq[:t]
        if v[t][partial_seq]==g[t][partial_seq]:
            return t
    return float('inf')

In [9]:
# now we calculate \tau
tau = {}
for coin_toss_seq in possible_coin_tosses(n=N):
    tau[coin_toss_seq] = min_n(V, G, coin_toss_seq)    
print(tau)    

{'HHH': inf, 'HHT': inf, 'HTH': inf, 'HTT': inf, 'THH': 1, 'THT': 1, 'TTH': 1, 'TTT': 1}


As we can see in the code above, $V_0=$ `V[0]['']` $=0.4$, and the optimal exercise poilicy is given by $\tau(\omega_1\omega_2\omega_3)=1$ whenver $\omega_1=T$, and $\tau(\omega_1\omega_2\omega_3)=\infty$ otherwise. $\square$